## 实验项目名称：决策树分类算法的肝病诊断

## 实验目的和要求：

- 掌握常见的数据读取方式
- 掌握 pandas 常用属性与方法
- 掌握 DecisionTreeClassifier 模型的构建及重要参数
- 掌握信息熵、信息增益、基尼系数的概念。
- 掌握分类模型评价的方法
- 掌握数据标准化的方法
- 掌握决策树的基本概念。

## 实验内容：

### 1.基于公开的肝病数据集，使用 sklearn 决策树算法构建对肝病分类预测模型

### 2.决策树可视化。

#### 数据来源

使用来自 UCI 的肝病数据集http://archive.ics.uci.edu/ml/datasets/ILPD+(Indian+Liver+Patient+Dataset) 。该数据集包含了 583 个数据样本，其中有 416 个患肝病，167 人未患肝病。每个数据样本共 11 个特征变量，其中一个特征变量表示是否患肝病，其中 1 表示患有肝病，2 表示未患肝病。


## 实验步骤及结果

#### 1.读入数据、探索数据


In [ ]:
import pandas as pd

row = pd.read_csv("./python_data/ILPD.csv", header=None)
row

从描述文件中得知数据共有 11 个特征及每个特征的含义，为了便于理解，将特征名翻译成中文之后给数据集添加上特征名称


In [ ]:
row.columns = [
    "年龄",
    "性别",
    "总胆红素",
    "直接胆红素",
    "Alkphos",
    "血糖",
    "血清",
    "总Protiens",
    "白蛋白",
    "A/G比值",
    "分类",
]

缺失值检查


In [ ]:
row.isnull().sum()

采用均值填补的方式，填补缺失值


In [ ]:
row["A/G比值"].fillna(value=row["A/G比值"].mean(), inplace=True)

重复值检查


In [ ]:
row.duplicated().sum()
row[row.duplicated()]
# row.drop_duplicates(inplace=True)
# row.reset_index()

将性别列中的字符更改为数字表示，以符合模型处理数据的类型要求，Female 转换为 0，Male 转换为 1


In [ ]:
list = []
for i in row["性别"]:
    if i == "Female":
        list.append(0)
    else:
        list.append(1)
row.insert(loc=2, column="Gender", value=list)
row.drop("性别", axis=1, inplace=True)
row

统计患病和未患病人数


In [ ]:
row["分类"].value_counts()

“年龄”特征的描述性统计情况


In [ ]:
row["年龄"].describe()

“总胆红素”和“直接胆红素”特征的描述性统计情况


In [ ]:
row.iloc[:, 2:4].describe()

#### 2.划分训练集和测试集


In [ ]:
from sklearn.model_selection import train_test_split

data = row.iloc[:, 0:10]
target = row.iloc[:, 10]
xtrain, xtest, ytrain, ytest = train_test_split(
    data, target, random_state=30, test_size=0.2
)

#### 3.选择信息增益划分属性建立决策树分类模型（可对重要参数 max_depth、min_samples_leaf & min_samples_split 等进行设置，优化模型分类效果）


In [ ]:
from sklearn import tree

clf2 = tree.DecisionTreeClassifier(
    criterion="entropy",
    random_state=30,
    splitter="random",
    max_depth=3,
    min_samples_leaf=10,
    min_samples_split=10,
)
clf2 = clf2.fit(xtrain, ytrain)

返回模型预测的准确度评分 score


In [ ]:
clf2.score(xtest, ytest)

决策树可视化


In [ ]:
import graphviz

dot_data = tree.export_graphviz(
    clf2,
    feature_names=row.columns[0:10],
    class_names=["患病", "未患病"],
    filled=True,
    rounded=True,
)
graph = graphviz.Source(dot_data)
graph

模型评估(混淆矩阵，分类评估报告)


In [ ]:
import numpy as np
from sklearn import metrics
from sklearn.metrics import classification_report

y_pred2 = clf2.predict(xtest)
metrics.accuracy_score(ytest, y_pred2)  # 模型的准确率
cnf_matrix2 = metrics.confusion_matrix(ytest, y_pred2)
print(cnf_matrix2)
print(classification_report(ytest, y_pred2))

#### 4.选择基尼指数划分属性建立决策树分类模型


In [ ]:
clf3 = tree.DecisionTreeClassifier(max_depth=3)
clf3 = clf3.fit(xtrain, ytrain)
dot_data = tree.export_graphviz(
    clf3,
    feature_names=row.columns[0:10],
    class_names=["患病", "未患病"],
    filled=True,
    rounded=True,
)

返回模型预测的准确度评分 score


In [ ]:
clf3.score(xtest, ytest)

决策树可视化


In [ ]:
graph = graphviz.Source(dot_data)
graph

模型评估(混淆矩阵，分类评估报告)


In [ ]:
y_pred3 = clf3.predict(xtest)
metrics.accuracy_score(ytest, y_pred3)  # 模型的准确率
cnf_matrix3 = metrics.confusion_matrix(ytest, y_pred3)
print(cnf_matrix3)
print(classification_report(ytest, y_pred3))

## 实验结果


1. 采用信息增益划分属性生成的决策树模型评分是多少？在属性划分过程中最重要的特征是哪一个？


- 采用信息增益划分属性生成的决策树模型评分是 0.7094017094017094
- 最重要的特征是总胆红素，由 graphviz 画出的决策树可以知道


2. 采用基尼系数划分属性生成的决策树模型评分是多少？在属性划分过程中最重要的特征是哪一个？


- 采用基尼系数划分属性生成的决策树模型评分是 0.7008547008547008
- 最重要的特征是直接胆红素，由 graphviz 画出的决策树可以知道


## 实验所需软件环境和库

Jupyter Notebook、Python、Sklearn、Numpy、Pandas、graphviz


## 教师评语
